In [ ]:
import numpy as np
from pandas_datareader import DataReader as pdr
import plotly.graph_objects as go

# EXAMPLE INPUTS

In [ ]:
start = 1980
end = 2022
nom_return = 0.1

# GET INFLATION DATA

In [ ]:
cpi = pdr('CPIAUCSL', 'fred', start=f"{start-1}-12-01", end=f"{end+1}-01-01")
cpi = cpi.resample('Y').last()

inflation = cpi.pct_change().iloc[1:-1]
inflation.columns = ["inflation"]
inflation.index = inflation.index.map(lambda x: x.year)
inflation.index.name = "year"
inflation.tail()

# FIGURE 1

In [ ]:
trace = go.Scatter(
    x=inflation.index.to_list(),
    y=inflation.inflation,
    mode="lines",
    hovertemplate="%{x}<br>inflation = %{y:.2%}<extra></extra>"
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Inflation Rate",
    yaxis_tickformat=".0%",
    template="plotly_white"
)
fig.show()

# HYPOTHETICAL REAL RETURNS

In [ ]:
real_return = (1+nom_return) / (1+inflation) - 1

# FIGURE 2

In [ ]:
trace1 = go.Scatter(
    x=real_return.index.to_list(),
    y=(1+nom_return)**np.arange(1, end-start+1),
    mode="lines",
    name="nominal $",
    hovertemplate="Accumulation in Nominal $ = %{y:0.2f}<extra></extra>"
)

trace2 = go.Scatter(
    x=real_return.index.to_list(),
    y=(1+real_return.iloc[:,0]).cumprod(),
    mode="lines",
    name="constant $",
    hovertemplate="Accumulation in Constant $ = %{y:0.2f}<extra></extra>"
)

fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Compound Return",
    yaxis_tickformat="$.0f",
    hovermode="x unified",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    template="plotly_white",
)
fig.show()